# Tutorial- Explore CNN architectures using arch_summary
> Exploring CNN architectures from torchvision, fastai2 and pytorch-image-models (timm)

In [ ]:
from fastai2.vision.all import *
from fast_impl.core import *

`arch_summary` function plays major role while deciding parameter groups for discriminative learning. It gives a brief summary of architecture and is independant of `DataLoaders` or `Learner`. Thus we could use this function to assess the correctness of architecture before passing it to learner.

## fastai2 models
Let's first quickly go through `XResNet` series offered by `fastai2`

```python
def xresnet18 (pretrained=False, **kwargs): return _xresnet(pretrained, 1, [2, 2,  2, 2], **kwargs)
```


In [ ]:
arch_summary(xresnet18)

(0 ) ConvLayer        : 3   layers
(1 ) ConvLayer        : 3   layers
(2 ) ConvLayer        : 3   layers
(3 ) MaxPool2d        : 1   layers
(4 ) Sequential       : 2   layers
(5 ) Sequential       : 2   layers
(6 ) Sequential       : 2   layers
(7 ) Sequential       : 2   layers
(8 ) AdaptiveAvgPool2d: 1   layers
(9 ) Flatten          : 1   layers
(10) Dropout          : 1   layers
(11) Linear           : 1   layers


Look at the Sequential layers from 4-7 all having two child layers, that's the meaning of `[2,2,2,2]` in the model definition. Let's go deeper and check what are those children

In [ ]:
arch_summary(xresnet18,verbose=True)

(0 ) ConvLayer        : 3   layers
      Conv2d
      BatchNorm2d
      ReLU
(1 ) ConvLayer        : 3   layers
      Conv2d
      BatchNorm2d
      ReLU
(2 ) ConvLayer        : 3   layers
      Conv2d
      BatchNorm2d
      ReLU
(3 ) MaxPool2d        : 1   layers
(4 ) Sequential       : 2   layers
      ResBlock
      ResBlock
(5 ) Sequential       : 2   layers
      ResBlock
      ResBlock
(6 ) Sequential       : 2   layers
      ResBlock
      ResBlock
(7 ) Sequential       : 2   layers
      ResBlock
      ResBlock
(8 ) AdaptiveAvgPool2d: 1   layers
(9 ) Flatten          : 1   layers
(10) Dropout          : 1   layers
(11) Linear           : 1   layers


hmm... those are indeed `ResBlocks`, but what is `ResBlock`?

In [ ]:
arch_summary(xresnet18,[4,0],verbose=True)

(0 ) Sequential       : 2   layers
      ConvLayer
      ConvLayer
(1 ) Sequential       : 0   layers
(2 ) ReLU             : 1   layers


![Residual Block](images/residual_block.png)

According to [ResNet](https://arxiv.org/abs/1512.03385), we should have two weight layers, an identity (skip-connection) block and activation (ReLU), which is exactly `ResBlock` implements, with many [tweaks](https://arxiv.org/abs/1812.01187) introduced in the following years to improve the performance. A `ConvLayer` in fastai is basically Conv2D --> BatchNorm --> Activation (ReLU) but we can infinitely customize this building block; you might want to have a look at its [implementation](https://github.com/fastai/fastai2/blob/a1204f7ba005fe92e09763f2b68dd9fb4ef11846/fastai2/layers.py#L230). For rest of `xresnet` family, we'll have the exact same architecture but with more "groups" of `ResBlock`, such as
```python
def xresnet34 (pretrained=False, **kwargs): return _xresnet(pretrained, 1, [3, 4,  6, 3], **kwargs)
def xresnet50 (pretrained=False, **kwargs): return _xresnet(pretrained, 4, [3, 4,  6, 3], **kwargs)
def xresnet101(pretrained=False, **kwargs): return _xresnet(pretrained, 4, [3, 4, 23, 3], **kwargs)
def xresnet152(pretrained=False, **kwargs): return _xresnet(pretrained, 4, [3, 8, 36, 3], **kwargs)
```

`xresnet34` will have 4 groups having `[3, 4, 6, 3]` no. of `ResBlocks` and so on. We do get other variants of these base architecutures but as they're still experimental, I'll skip them for now. Now let's have a look at some architectures from torchvision. I've already explained `densenet121` in [core](core.ipynb) notebook

## torchvision models